In [16]:
import pandas as pd
import numpy as np
import zipfile

In [17]:
!unzip new_data.zip

Archive:  new_data.zip
replace new_data/a.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import LSTM, Dense,Conv1D,MaxPooling1D,Flatten,Dropout,GlobalAveragePooling1D
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam

# Function to preprocess the data
def preprocess_data(df):
    groups = df.groupby('Output')
    sequences = []
    for name, group in groups:
        indices = group['Output'].values
        change_indices = np.where(indices[:-1] != indices[1:])[0] + 1
        if len(change_indices) > 0:
            sub_sequences = np.split(group.iloc[:, 1:], change_indices)
        else:
            sub_sequences = [group.iloc[:, 1:]]
        sequences.extend(sub_sequences)  # Append sequences to the list
    return sequences

# Load data and preprocess
l = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()
all_sequences = []
labels = []

for letter in l:
    filename = f"/content/new_data/{letter}.csv"
    df = pd.read_csv(filename)
    df.drop(0,axis=0,inplace=True)
    sequences = preprocess_data(df)
    all_sequences.extend(sequences)  # Extend the list with sequences from the current file
    labels.extend([letter] * len(sequences))  # Assign labels for sequences from the current file

# Pad sequences to ensure they have the same length
max_length = max([len(seq) for seq in all_sequences])
padded_sequences = pad_sequences(all_sequences, maxlen=max_length, padding='post', dtype='float32')

# Convert sequences and labels to numpy arrays
X = np.array(padded_sequences)
y = np.array(labels)

# Shuffle sequences and labels simultaneously while maintaining correspondence
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(y)

y_categorical = to_categorical(y_encoded)

train_indices=[]
test_indices=[]
for i in range(len(X)):
  if i%5==0:
    test_indices.append(i)
  else:
    train_indices.append(i)
np.random.shuffle(train_indices)

X_test,y_test,X_train,y_train=X[test_indices],y_categorical[test_indices],X[train_indices],y_categorical[train_indices]

In [19]:
# Define the model
model = Sequential()

model.add(Conv1D(32,kernel_size=2,strides=1,activation='gelu'))
model.add(Conv1D(32,kernel_size=2,strides=1,activation='gelu'))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Conv1D(32,kernel_size=2,strides=1,activation='gelu'))
model.add(Conv1D(32,kernel_size=2,strides=1,activation='gelu'))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Flatten())

model.add(Dense(64,activation='gelu'))

model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Change this line

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=25, batch_size=32,verbose=1)

# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

# Make predictions
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Decode predicted labels
y_pred_decoded = label_encoder.inverse_transform(y_pred_labels)

# Calculate accuracy
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_labels)
print("Test Accuracy:", accuracy)


Epoch 1/25
4/4 [==============================] - 2s 132ms/step - loss: 3.6585 - accuracy: 0.0385 - val_loss: 3.3799 - val_accuracy: 0.0385
Epoch 2/25
4/4 [==============================] - 0s 40ms/step - loss: 2.5073 - accuracy: 0.3173 - val_loss: 3.2284 - val_accuracy: 0.0769
Epoch 3/25
4/4 [==============================] - 0s 46ms/step - loss: 1.9150 - accuracy: 0.6442 - val_loss: 3.1519 - val_accuracy: 0.1923
Epoch 4/25
4/4 [==============================] - 0s 45ms/step - loss: 1.2844 - accuracy: 0.8173 - val_loss: 3.2317 - val_accuracy: 0.2308
Epoch 5/25
4/4 [==============================] - 0s 65ms/step - loss: 0.7277 - accuracy: 0.8942 - val_loss: 3.4425 - val_accuracy: 0.1923
Epoch 6/25
4/4 [==============================] - 0s 65ms/step - loss: 0.3361 - accuracy: 0.9615 - val_loss: 3.6996 - val_accuracy: 0.1538
Epoch 7/25
4/4 [==============================] - 0s 65ms/step - loss: 0.1310 - accuracy: 1.0000 - val_loss: 4.0666 - val_accuracy: 0.1923
Epoch 8/25
4/4 [==========

In [20]:
model.save('model_1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
